In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import string
import nltk
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score


from subprocess import check_output

In [3]:
# Import the email modules we'll need
import glob
import email
import mailparser
from email import policy
from email.parser import BytesParser

path = '../datawe/raw/Email_Classification/*'
email_types = glob.glob(path)
appendFilesData = []
file_raw_data = [] 
for folder in email_types:
    files = glob.glob(folder+"/*.txt")
    email_type = folder.split('\\')[1]
    for name in files:
        try:
            with open(name) as fp:
                raw_data = fp.read()
                file_raw_data.append(raw_data)
                msg = mailparser.parse_from_string(raw_data)
                appendFilesData.append({
                    "to":msg.to,
                    "from":msg.from_,
                    "subject":msg.subject,
                    "date":msg.date,
#                     "sent":msg["Sent"],
#                     "importance":msg["Importance"],
                    "content":  msg.body,
                    "class_to_exec":email_type,
                })
         
        except IOError as exc:
            print('Exception')

            
#creating pandas dataframe
data = pd.DataFrame(appendFilesData)

In [14]:
data

,class_to_exec,content,date,from,subject,to
0,Booking,,None,[],,[]
1,Booking,"Dear Rohit,\n \nAs per our conversation a whil...",None,"[(, Meemendra), (, ), (, meemendra.kumar@andbe...",Mr. Nagdev X 04,"[(, Gateway)]"
2,Booking,75676SB039041\n\n \n\nDear Sir/Madam\n\n \n\nR...,None,"[(, Amrit), (, ), (, amrits@optusnet.com.au)]",Re: Taj PalaceNew Delhi Payment link for reser...,"[(, Reservations)]"
3,Booking,"As per corporate holiday plan, kindly arrange ...",None,"[(, tehp@tatasteel.com[mailto:tehp@tatasteel.c...",TEHP/18/12809: Booking request of Arun Gadamsh...,"[(Reservations Coorg, reservations.coorg@tajho..."
4,Booking,"Dear Gampa ji,\n \nGreetings of the day !\n \n...",None,"[(, INDIAN), (, ), (, indianjourney@gmail.com)]",Valley Period/3 double room/Lerner group,"[(, Neeraj)]"
5,Booking,"Dear Reservation Dept,\n \nGreetings from Kual...",None,"[(, Shazana), (, ), (, sabdulkhalid@ippfeseaor...","Vivanta Ambassador, New Delhi - ROOM BOOKING","[(Reservations, Reservations@tajhotels.com)]"
6,Cancellation,,None,[],,[]
7,Cancellation,,None,[],,[]
8,Cancellation,,None,[],,[]
9,Cancellation,Hi\n\nPlease cancel my booking .,None,"[(, rajivbhatia@ase.ae[mailto:rajivbhatia@ase....",Re: Taj Chandigarh Payment link for reservatio...,"[(Reservations, Reservations@tajhotels.com)]"


In [20]:
##text cleaning
def clean(doc):
    words_to_exclude = set(stopwords.words('english'))
    exclude = set(string.punctuation)
    
    doc = doc.replace('\n', ' ')
    tagged_sentence = nltk.tag.pos_tag(doc.split())
    edited_sentence = ' '.join([word for word,tag in tagged_sentence if tag != 'NNP' and tag != 'NNPS'])

    word_free = " ".join([i for i in edited_sentence.lower().split() if i not in words_to_exclude])
    punc_free = ''.join(ch for ch in word_free if ch not in exclude)
    alpha_free = " ".join(word for word in punc_free.split() if word.isalpha())
    #     stemmer = SnowballStemmer("english").stem
    #     stem_free = " ".join(stemmer(stem) for stem in alpha_free.split())
    lemma = WordNetLemmatizer()
    normalized = " ".join(lemma.lemmatize(word) for word in alpha_free.split())
    
#     print(edited_sentence)
#     print(word_free)
#     print(punc_free)
#     print(alpha_free)
#     print(stem_free)
    

    return normalized



In [16]:
##text cleaning
def NounsAndPluralNouns(doc):
    words_to_exclude = set(stopwords.words('english'))
    exclude = set(string.punctuation)
    
    
    tagged_sentence = nltk.tag.pos_tag(doc.split())
    edited_sentence = ' '.join([word for word,tag in tagged_sentence if tag == 'NNP' or tag == 'NNPS'])

#     word_free = " ".join([i for i in edited_sentence.lower().split() if i not in words_to_exclude])
#     punc_free = ''.join(ch for ch in word_free if ch not in exclude)
#     alpha_free = " ".join(word for word in punc_free.split() if word.isalpha())
#     #     stemmer = SnowballStemmer("english").stem
#     #     stem_free = " ".join(stemmer(stem) for stem in alpha_free.split())
#     lemma = WordNetLemmatizer()
#     normalized = " ".join(lemma.lemmatize(word) for word in alpha_free.split())
    
    print(tagged_sentence)
#     print(word_free)
#     print(punc_free)
#     print(alpha_free)
#     print(stem_free)
    

    return edited_sentence



### removing nouns from text

In [17]:
NounsAndPluralNouns(""" 
Dear Sirs,
 
Please raise your final invoice on :
 
Macawber Beekay Pvt Ltd 
C/O 4th Floor KB-25,Salt Lake City 
Kolkata 700098
West Bengal
19AAECB8838L2Z0
 
Thanking you,
 
Regards,
 
Ajay Kumar Gupta""").split()

[('Dear', 'NNP'), ('Sirs,', 'NNP'), ('Please', 'NNP'), ('raise', 'VB'), ('your', 'PRP$'), ('final', 'JJ'), ('invoice', 'NN'), ('on', 'IN'), (':', ':'), ('Macawber', 'NNP'), ('Beekay', 'NNP'), ('Pvt', 'NNP'), ('Ltd', 'NNP'), ('C/O', 'NNP'), ('4th', 'CD'), ('Floor', 'NNP'), ('KB-25,Salt', 'NNP'), ('Lake', 'NNP'), ('City', 'NNP'), ('Kolkata', 'NNP'), ('700098', 'CD'), ('West', 'NNP'), ('Bengal', 'NNP'), ('19AAECB8838L2Z0', 'CD'), ('Thanking', 'NNP'), ('you,', 'NN'), ('Regards,', 'NNP'), ('Ajay', 'NNP'), ('Kumar', 'NNP'), ('Gupta', 'NNP')]


['Dear',
 'Sirs,',
 'Please',
 'Macawber',
 'Beekay',
 'Pvt',
 'Ltd',
 'C/O',
 'Floor',
 'KB-25,Salt',
 'Lake',
 'City',
 'Kolkata',
 'West',
 'Bengal',
 'Thanking',
 'Regards,',
 'Ajay',
 'Kumar',
 'Gupta']

In [18]:
# remove_nouns(data['content'][0])

In [21]:
## text transform 
textFeatures = data['content'].copy()
textFeatures = textFeatures.apply(clean)
vectorizer =  TfidfVectorizer(token_pattern=r"(?u)\b\w+\b", stop_words=None,  analyzer='word', norm=None)#TfidfVectorizer("english") 
features = vectorizer.fit_transform(textFeatures)

features_train, features_test, labels_train, labels_test = train_test_split(features, data['class_to_exec'], test_size=0.2, random_state=111)

In [28]:
# import gensim

# dictionary = gensim.corpora.Dictionary(trn_set)
# dictionary.filter_extremes(no_below=20, no_above=0.1)

# print(dictionary)

In [ ]:
trn_set = pd.DataFrame(' '.join(textFeatures.as_matrix()).split())[0].value_counts().keys()

In [ ]:
vectorizer.use_idf

In [ ]:
help(vectorizer)

In [22]:
from sklearn.naive_bayes import MultinomialNB

mnb = MultinomialNB(alpha=0.2)
mnb.fit(features_train, labels_train)
prediction = mnb.predict(features_test)
accuracy_score(labels_test,prediction)

0.3333333333333333

In [23]:
## grid serach CV for tuning hyperparameters
import sklearn.model_selection as modsel
from sklearn.linear_model import LogisticRegression
param_grid_ = {'C': [1e-5, 1e-3, 1e-1, 1e0, 1e1, 1e2]}

tfidf_search = modsel.GridSearchCV(LogisticRegression(), cv=5, param_grid=param_grid_)
tfidf_search.fit(features_train, labels_train)

c:\users\lokesh\appdata\local\programs\python\python36\lib\site-packages\sklearn\model_selection\_split.py:626: Warning: The least populated class in y has only 4 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
c:\users\lokesh\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\lokesh\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:459: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
c:\users\lokesh\appdata\local\programs\python\python36\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from 

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'C': [1e-05, 0.001, 0.1, 1.0, 10.0, 100.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [24]:
pre = tfidf_search.predict(features_test)
accuracy_score(labels_test,pre)

0.3333333333333333

In [25]:
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC

svc = SVC(kernel='sigmoid', gamma=1.0)
svc.fit(features_train, labels_train)
prediction = svc.predict(features_test)
accuracy_score(labels_test,prediction)

0.16666666666666666

In [26]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(features_train, labels_train)
prediction = lr.predict(features_test)
accuracy_score(labels_test,prediction)

0.3333333333333333

In [27]:
help(vectorizer)

Help on TfidfVectorizer in module sklearn.feature_extraction.text object:

class TfidfVectorizer(CountVectorizer)
 |  Convert a collection of raw documents to a matrix of TF-IDF features.
 |  
 |  Equivalent to CountVectorizer followed by TfidfTransformer.
 |  
 |  Read more in the :ref:`User Guide <text_feature_extraction>`.
 |  
 |  Parameters
 |  ----------
 |  input : string {'filename', 'file', 'content'}
 |      If 'filename', the sequence passed as an argument to fit is
 |      expected to be a list of filenames that need reading to fetch
 |      the raw content to analyze.
 |  
 |      If 'file', the sequence items must have a 'read' method (file-like
 |      object) that is called to fetch the bytes in memory.
 |  
 |      Otherwise the input is expected to be the sequence strings or
 |      bytes items are expected to be analyzed directly.
 |  
 |  encoding : string, 'utf-8' by default.
 |      If bytes or files are given to analyze, this encoding is used to
 |      decode.
 

In [93]:
features.toarray().shape

(24, 183)

In [161]:
1e1*0.1

1.0

In [188]:
!pip install spacy

  Running setup.py bdist_wheel for ujson: started
  Running setup.py bdist_wheel for ujson: finished with status 'error'
  Complete output from command c:\users\lokesh\appdata\local\programs\python\python36\python.exe -u -c "import setuptools, tokenize;__file__='C:\\Users\\Lokesh\\AppData\\Local\\Temp\\pip-install-500wzv43\\ujson\\setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" bdist_wheel -d C:\Users\Lokesh\AppData\Local\Temp\pip-wheel-i9k0_2q9 --python-tag cp36:
  running bdist_wheel
  running build
  running build_ext
  building 'ujson' extension
  error: Microsoft Visual C++ 14.0 is required. Get it with "Microsoft Visual C++ Build Tools": https://visualstudio.microsoft.com/downloads/
  
  ----------------------------------------
  Running setup.py clean for ujson
  Running setup.py bdist_wheel for dill: started
  Running setup.py bdist_wheel for dill: finished with status 'done'
  Stored in 

  Failed building wheel for ujson
  Failed building wheel for cytoolz
Command "c:\users\lokesh\appdata\local\programs\python\python36\python.exe -u -c "import setuptools, tokenize;__file__='C:\\Users\\Lokesh\\AppData\\Local\\Temp\\pip-install-500wzv43\\ujson\\setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" install --record C:\Users\Lokesh\AppData\Local\Temp\pip-record-fdohh__7\install-record.txt --single-version-externally-managed --compile" failed with error code 1 in C:\Users\Lokesh\AppData\Local\Temp\pip-install-500wzv43\ujson\


In [190]:
import re
regex = r"([a-zA-Z]+) (\d+)" 
text = "June 24" 
match = re.search(regex, text) 
if match:
    #   match.group() or match.group(0) always returns the fully matched string i.e. "June 24"     
    print("Match: %s" % (match.group(0)))
    #   match.group(1) match.group(2), ... will return the capture groups in order
    print("Month: %s" % (match.group(1))) # "June"     
    print("Day: %s" % (match.group(2))) # "24" else:
    # If re.search() does not match, then None is returned     
    print("The regex pattern does not match. :(")

Match: June 24
Month: June
Day: 24
The regex pattern does not match. :(


In [195]:
help(dir)

Help on built-in function dir in module builtins:

dir(...)
    dir([object]) -> list of strings
    
    If called without an argument, return the names in the current scope.
    Else, return an alphabetized list of names comprising (some of) the attributes
    of the given object, and of attributes reachable from it.
    If the object supplies a method named __dir__, it will be used; otherwise
    the default dir() logic is used and returns:
      for a module object: the module's attributes.
      for a class object:  its attributes, and recursively the attributes
        of its bases.
      for any other object: its attributes, its class's attributes, and
        recursively the attributes of its class's base classes.



In [207]:
from nltk import ne_chunk, pos_tag, word_tokenize
from nltk.tree import Tree


def get_continuous_chunks(text):
    chunked = ne_chunk(pos_tag(word_tokenize(text)))
    continuous_chunk = []
    current_chunk = []
    for i in chunked:
        if type(i) == Tree:
            current_chunk.append(" ".join([token for token, pos in i.leaves()]))
        elif current_chunk:
            named_entity = " ".join(current_chunk)
            if named_entity not in continuous_chunk:
                continuous_chunk.append(named_entity)
                current_chunk = []
            else:
                continue
    return continuous_chunk

my_sent = """ 
Dear Sirs,
 
Please raise your final invoice on :
 
Macawber Beekay Pvt Ltd 
C/O 4th Floor KB-25,Salt Lake City 
Kolkata 700098
West Bengal
19AAECB8838L2Z0
 
Thanking you,
 
Regards,
 
Ajay Kumar Gupta"""
get_continuous_chunks(text)


['Subject',
 'Arun Gadamshetty',
 'Vivanta',
 'Taj',
 'Madikeri Coorg',
 'Coorg',
 'Arun Gadamshetty Spares Manufacturing',
 'Taj Madikeri Coorg',
 'Coorg Room Type',
 'Superior',
 'Honeymoon',
 'Traveller Details',
 'Arun Gadamshetty SHWETA',
 'Spouse Age']

In [217]:
text = """

Serial number : TEHP/18/12809
Name : Arun Gadamshetty
P No. : 153021
Department : Spares Manufacturing Department
Contact No : 8092084556
Place : Vivanta by Taj - Madikeri Coorg, Coorg
Room Type : Superior Charm
Check In Date-Time : 10 Dec 2018 02:00 PM
Check Out Date-Time : 12 Dec 2018 12:00 PM
No of rooms :1
Comments by requester : For Honeymoon trip

Traveller Details: 

1. 
    Name : Arun Gadamshetty
    Relation : Self
    Age : 34 years
2. 
    Name : SHWETA GADAMSHETTY
    Relation : Spouse
    Age : 28 years

"""





In [218]:
from nltk import ne_chunk, pos_tag, word_tokenize
from nltk.tree import Tree

def get_continuous_chunks(text):
    chunked = ne_chunk(pos_tag(word_tokenize(text)))
    prev = None
    continuous_chunk = []
    current_chunk = []

    for i in chunked:
        if type(i) == Tree:
            current_chunk.append(" ".join([token for token, pos in i.leaves()]))
        elif current_chunk:
            named_entity = " ".join(current_chunk)
            if named_entity not in continuous_chunk:
                continuous_chunk.append(named_entity)
                current_chunk = []
        else:
            continue

    if continuous_chunk:
        named_entity = " ".join(current_chunk)
        if named_entity not in continuous_chunk:
            continuous_chunk.append(named_entity)
    print(chunked)

    return continuous_chunk

txt = "Barack Obama is a great person." 
print(get_continuous_chunks(text))

(S
  (GPE Serial/JJ)
  number/NN
  :/:
  TEHP/18/12809/JJ
  Name/NN
  :/:
  (PERSON Arun/NNP Gadamshetty/NNP)
  P/NNP
  No/NNP
  ./.
  :/:
  153021/CD
  Department/NNP
  :/:
  (PERSON Spares/NNS Manufacturing/NNP)
  Department/NNP
  Contact/NNP
  No/NNP
  :/:
  8092084556/CD
  Place/NN
  :/:
  Vivanta/VBN
  by/IN
  (ORGANIZATION Taj/NNP)
  -/:
  (PERSON Madikeri/NNP Coorg/NNP)
  ,/,
  (PERSON Coorg/NNP Room/NNP Type/NNP)
  :/:
  (ORGANIZATION Superior/JJ)
  Charm/NNP
  Check/NNP
  In/IN
  Date-Time/NNP
  :/:
  10/CD
  Dec/NNP
  2018/CD
  02:00/CD
  PM/NNP
  Check/NNP
  Out/IN
  Date-Time/NNP
  :/:
  12/CD
  Dec/NNP
  2018/CD
  12:00/CD
  PM/NNP
  No/NNP
  of/IN
  rooms/NNS
  :1/JJ
  Comments/NNS
  by/IN
  requester/NN
  :/:
  For/IN
  (GPE Honeymoon/NNP)
  trip/NN
  (PERSON Traveller/NNP Details/NNP)
  :/:
  1/CD
  ./.
  Name/NN
  :/:
  (PERSON Arun/NNP Gadamshetty/NNP)
  Relation/NNP
  :/:
  Self/JJ
  Age/NNP
  :/:
  34/CD
  years/NNS
  2/CD
  ./.
  Name/NN
  :/:
  (ORGANIZATION SHWET

In [37]:
import spacy

In [38]:
spacy.load('en')

OSError: [E050] Can't find model 'en'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.

In [1]:
import spacy

In [40]:
spacy.load('en')

OSError: [E050] Can't find model 'en'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.

In [34]:
import re
import nltk
import codecs
import string
import subprocess
import unicodedata

BASE = os.path.join(os.path.dirname('C:\Users\Lokesh\AppData\Roaming\nltk_data'), "..")
# Create an NLTK corpus reader to access text data on disk.                
CORPUS = os.path.join(BASE, "data", "corpus")

kddcorpus = nltk.corpus.PlaintextCorpusReader(CORPUS, '.*\.txt')

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (<ipython-input-34-03fa0a9cf1d3>, line 8)

In [31]:
nltk.data.path

['C:\\Users\\Lokesh/nltk_data',
 'c:\\users\\lokesh\\appdata\\local\\programs\\python\\python36\\nltk_data',
 'c:\\users\\lokesh\\appdata\\local\\programs\\python\\python36\\share\\nltk_data',
 'c:\\users\\lokesh\\appdata\\local\\programs\\python\\python36\\lib\\nltk_data',
 'C:\\Users\\Lokesh\\AppData\\Roaming\\nltk_data',
 'C:\\nltk_data',
 'D:\\nltk_data',
 'E:\\nltk_data']

In [4]:
from collections import defaultdict
from nltk import ne_chunk
# from polyglot.text import Text
from nltk.tag import StanfordNERTagger

# def polyglot_entities(fileids=None, section = None, corpus=kddcorpus):
#     """
#     Extract entities from each file using polyglot
#     """
#     results = defaultdict(lambda: defaultdict(list))
#     fileids = fileids or corpus.fileids()
#     for fileid in fileids:
#         if section is not None:
#             text = Text((list(sectpull([fileid],section=section))[0][1]))
#         else:
#             text = Text(corpus.raw(fileid))
#         for entity in text.entities:
#             etext = " ".join(entity)
#             if entity.tag == 'I-PER':
#                 key = 'persons'
#             elif entity.tag == 'I-ORG':
#                 key = 'organizations'
#             elif entity.tag == 'I-locations':
#                 key = 'locations'
#             else:
#                 key = 'other'
#             results[fileid][key].append(etext)
#     return results
def stanford_entities(model, jar, fileids=None, corpus=kddcorpus, section = None):
    """
    Extract entities using the Stanford NER tagger.
    Must pass in the path to the tagging model and jar as downloaded from the
    Stanford Core NLP website.
    """
    results = defaultdict(lambda: defaultdict(list))
    fileids = fileids or corpus.fileids()
    tagger  = StanfordNERTagger(model, jar)
    section = section
    for fileid in fileids:
        if section is not None:
            text = nltk.word_tokenize(list(sectpull([fileid],section=section))[0][1])
        else:
            text  = corpus.words(fileid)
        chunk = []
        for token, tag in tagger.tag(text):
            if tag == 'O':
                if chunk:
                    # Flush the current chunk
                    etext =  " ".join([c[0] for c in chunk])
                    etag  = chunk[0][1]
                    chunk = []
                    if etag == 'PERSON':
                        key = 'persons'
                    elif etag == 'ORGANIZATION':
                        key = 'organizations'
                    elif etag == 'LOCATION':
                        key = 'locations'
                    else:
                        key = 'other'
                    results[fileid][key].append(etext)
            else:
                # Build chunk from tags
                chunk.append((token, tag))
    return results

def nltk_entities(fileids=None, section = None,corpus=kddcorpus):
    """
    Extract entities using the NLTK named entity chunker.
    """
    results = defaultdict(lambda: defaultdict(list))
    fileids = fileids or corpus.fileids()
    for fileid in fileids:
        if section is not None:
            text = nltk.pos_tag(nltk.word_tokenize(list(sectpull([fileid],section=section))[0][1]))
        else:
            text = nltk.pos_tag(corpus.words(fileid))
        for entity in nltk.ne_chunk(text):
            if isinstance(entity, nltk.tree.Tree):
                etext = " ".join([word for word, tag in entity.leaves()])
                label = entity.label()
            else:
                continue
            if label == 'PERSON':
                key = 'persons'
            elif label == 'ORGANIZATION':
                key = 'organizations'
            elif label == 'LOCATION':
                key = 'locations'
            elif label == 'GPE':
                key = 'other'
            else:
                key = None
            if key:
                results[fileid][key].append(etext)
    return results

NameError: name 'kddcorpus' is not defined

In [35]:
# Only extract our annotated files.
fids  = ['p19.txt', 'p29.txt']
# NLTK Entities
nltkents = nltk_entities(fids, section='top')
# Polyglot Entities
polyents = polyglot_entities(fids, section='top')
# Stanford Model Loading
root  = os.path.expanduser('~/models/stanford-ner-2014-01-04/')
model = os.path.join(root, 'classifiers/english.muc.7class.distsim.crf.ser.gz')
jar   = os.path.join(root, 'stanford-ner-2014-01-04.jar')
# Stanford Entities
stanents = stanford_entities(model, jar, fids, section='top')

NameError: name 'nltk_entities' is not defined

In [36]:
nlp = spacy.load('en')

OSError: [E050] Can't find model 'en'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.

In [ ]:
!python -m spacy download en_core_web_md